In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/var/folders/c4/kvf3th_j15lf5cq2pybvfgjm0000gn/T/ipykernel_74361/3590037574.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/Users/rhy/Library/CloudStorage/OneDrive-ArizonaStateUniversity/SelfStudy/Agentic & Gen AI with Cloud/agenticai/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

In [3]:
len(embeddings.embed_query("Hello world"))

384

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [5]:
len(embeddings.embed_query("Hello world"))

768

In [6]:
from pinecone import Pinecone

os.environ['PINECONE_API_KEY'] = os.getenv("PINECONE_API_KEY")

In [7]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [8]:
from pinecone import ServerlessSpec

#### Servlerless means that the server will be managed by the cloud Provider

In [9]:
index_name = "rhy-genai"

In [10]:
pc.has_index(index_name)

True

### Creating an Index

In [11]:
if not pc.has_index(index_name):
    pc.create_index(name=index_name, dimension=768, metric="cosine", spec=ServerlessSpec(cloud="aws",region="us-east-1"))

### Loading the index

In [12]:
index = pc.Index(index_name)

In [13]:
from langchain_pinecone import PineconeVectorStore

In [14]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [15]:
vector_store.similarity_search("What is Langchain?", k=3)

[Document(id='fcc31f4e-b144-4b0d-a435-b56721dcdcad', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='1fd7ead9-bea1-409c-bca6-0b3cd81a7b17', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='7f4b9f85-8bc7-4180-a961-404cee9a45c0', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]

In [16]:
from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.", metadata={"source": "tweet"})

document_2 = Document(page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.", metadata={"source": "news"})

document_3 = Document(page_content="Building an exciting new project with LangChain - come check it out!", metadata={"source": "tweet"})

document_4 = Document(page_content="Robbers broke into the city bank and stole $1M in cash", metadata={"source": "news"})

document_5 = Document(page_content="wow! That was an amazing movie. I can't wait to see it again.",metadata={"source": "tweet"})

document_6 = Document(page_content="Is the new iPhone worth the price? Read this review to find out.", metadata={"source":"website"})

document_7 = Document(page_content="The top 10 soccer players in the world right now.", metadata={"source": "website"})

document_8 = Document(page_content="LangGraph is the best framework for building stateful, agentic applications!", metadata={"source": "tweet"})

document_9 = Document(page_content="The stock market is down 500 points today due to fears of a recession.",metadata={"source": "news"})

document_10 = Document(page_content="I have a bad feeling I am going to get deleted :(", metadata={"source": "tweet"})

documents = [document_1, document_2, document_3, document_4, document_5, document_6, document_7, document_8, document_9, document_10]

In [17]:
uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents, ids=uuids)

['8225dbec-4d94-421c-9c7c-2d60d84ce024',
 '4d6c4070-4835-4551-98d2-7a4bc0719c2b',
 '3ff53cf4-7922-47a2-a004-6c0b2893635a',
 'eadef95d-46b6-4ebd-a844-8d1276f481e5',
 '31507cf0-ccf6-4e2d-a210-13f10030dc29',
 '3ff8cdb2-d534-4648-8f8e-05913adbe269',
 '845f4220-0ef4-484c-8a3d-0186348270c4',
 '931da4b7-27bd-4cba-b3de-95b658040b23',
 '7bf58bad-c649-4747-9664-7bfb7a4f9d3d',
 '577fa96f-ba39-4187-ab2e-0a39074017bb']

In [18]:
vector_store.similarity_search("What is Langchain?", k=3)

[Document(id='fcc31f4e-b144-4b0d-a435-b56721dcdcad', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='1fd7ead9-bea1-409c-bca6-0b3cd81a7b17', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='3ff53cf4-7922-47a2-a004-6c0b2893635a', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!')]

In [19]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [20]:
retriever.invoke("What is Langchain?")

[Document(id='fcc31f4e-b144-4b0d-a435-b56721dcdcad', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='1fd7ead9-bea1-409c-bca6-0b3cd81a7b17', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='3ff53cf4-7922-47a2-a004-6c0b2893635a', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!')]

In [29]:
from  langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [30]:
import pprint
pprint.pprint(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [32]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [33]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [36]:
rag_chain = ({"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt | model | StrOutputParser())

NameError: name 'model' is not defined

In [37]:
rag_chain.invoke("What is Langchain?")

NameError: name 'rag_chain' is not defined